# Vote prediction

Your friend runs an evil government and he wants to influence elections in a foreign country. His evil spy network collected data about voters in a foreign country:

- 1 if a voter older then 35 years, 0 otherwise
- 1 if a voter male, 0 otherwise
- 1 if a voter watched PythonNN in last month, 0 otherwise
- 1 if a voter watched Rabbit News in last month, 0 otherwise
- 1 if a voter lives in a big city, 0 otherwise
- 1 if a voter voted last time, 0 otherwise
- 1 if a voter likes ice cream, 0 otherwise
- 1 if a voter has hair, 0 otherwise

An evil plan of your friend is following:

- Based on 8 features predict how a person will vote
- Model if watching Rabbit News influences voters to vote for needed option
- Go to national parks and feed the rabbits
- A population of rabbits will grow, more people will see them in park
- People who will see rabbits in a park will decide to watch Rabbit News

Your friend just notified you that they were able to collect information about voters, but they were not able to get information on how people voted before because that country employs secret vote system.

They have information on how people voted in aggregate but not on voter level.
So, now it is your work to help your evil government and earn a hero status.
You are given data from previous elections. 8*(number of voters) features and binary result:

- 1 if more then half voters voted in favor, 0 otherwise

Your task is based on this information to predict how people will vote on next election.

# Setup

Copy auxiliary files from GitHub 

In [0]:
!wget https://raw.githubusercontent.com/VVKot/mlinseconds-vote-prediction/master/mlis/utils/gridsearch.py -q
!wget https://raw.githubusercontent.com/VVKot/mlinseconds-vote-prediction/master/mlis/utils/solutionmanager.py -q
!wget https://raw.githubusercontent.com/VVKot/mlinseconds-vote-prediction/master/mlis/utils/speedtest.py -q

Import libraries and utils

In [0]:
!pip3 install tensorboardX

In [0]:
import time
import random
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import solutionmanager as sm
from gridsearch import GridSearch

Check whether CUDA is available

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Create neural network

In [0]:
class SolutionModel(nn.Module):
    def __init__(self, input_size, output_size, solution):
        super(SolutionModel, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.solution = solution
        self.learning_rate = solution.learning_rate
#         self.momentum = solution.momentum
        self.hidden_size = solution.hidden_size
        self.activation_hidden = solution.activation_hidden
        self.activation_output = nn.Sigmoid()
        self.do_batch_norm = solution.do_batch_norm
        self.layers_number = solution.layers_number
        if self.solution.grid_search.enabled:
            torch.manual_seed(solution.random)
        self.hidden_size = self.solution.hidden_size
        self.linears = nn.ModuleList([nn.Linear(self.input_size if i == 0 else self.hidden_size, self.hidden_size if i != self.solution.layers_number -1 else self.output_size) for i in range(self.solution.layers_number)]).to(device)
        self.batch_norms = nn.ModuleList([nn.BatchNorm1d(self.hidden_size if i != self.solution.layers_number-1 else self.output_size, track_running_stats=False) for i in range(self.solution.layers_number)]).to(device)

    def forward(self, x):
        for i in range(len(self.linears)):
            x = self.linears[i](x)
            if self.solution.do_batch_norm:
                x = self.batch_norms[i](x)
            if i == len(self.linears)-1:
                x = self.activation_output(x)
            else:            
                x = self.solution.activations[self.activation_hidden](x)
        return x

    def calc_loss(self, output, target):
        bce_loss = nn.BCELoss()
        loss = bce_loss(output, target)
        return loss

    def calc_predict(self, output):
        predict = output.round()
        return predict

Create class to store hyper parameters. Implement grid search

In [0]:
class Solution():
    def __init__(self):
        self.best_step = 1000
        self.activations = {
            'relu': nn.ReLU(),
            'hardshrink': nn.Hardshrink(1),
            'relu6': nn.ReLU6(),
            'leakyrelu01': nn.LeakyReLU(0.1),
            'leakyrelu001': nn.LeakyReLU(0.01)
        }
        self.learning_rate = 0.001
#         self.momentum = 0.9
        self.hidden_size = 40
        self.layers_number = 10
        self.activation_hidden = 'leakyrelu01'
        self.activation_output = 'sigmoid'
        self.do_batch_norm = True
        self.sols = {}
        self.solsSum = {}
        self.random = 0
        self.random_grid = [_ for _ in range(10)]
        self.layers_number_grid = [5, 10, 15, 20]
#         self.hidden_size_grid = [20, 25, 28, 30, 32, 35, 38, 40, 45]
        self.hidden_size_grid = [5, 10, 15, 20, 30, 40, 50]
#         self.momentum_grid = [0.0, 0.3, 0.5, 0.8, 0.9]
#         self.learning_rate_grid = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.2, 1.5]
        self.learning_rate_grid = [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]
        self.activation_hidden_grid = list(self.activations.keys())
        self.grid_search = GridSearch(self)
        self.grid_search.set_enabled(True)

    def create_model(self, input_size, output_size):
        return SolutionModel(input_size, output_size, self)

    def get_key(self):
        return "{}_{}_{}_{}_{}_{}".format(self.learning_rate, self.hidden_size, self.activation_hidden, self.activation_output, self.do_batch_norm, "{0:03d}".format(self.layers_number));

    # Return number of steps used
    def train_model(self, model, train_data, train_target, context):
        key = self.get_key()
        if key in self.sols and self.sols[key] == -1:
            return
        step = 0
        model.to(device)
        # Put model in train mode
        model.train()
#         optimizer = optim.SGD(model.parameters(), lr=self.learning_rate, momentum=self.momentum)
        optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)
        while True:
            time_left = context.get_timer().get_time_left()
            # No more time left, stop training
            if time_left < 0.1:
                break
            data = train_data
            target = train_target
            # model.parameters()...gradient set to zero
            optimizer.zero_grad()
            # evaluate model => model.forward(data)
            output = model(data)
            # if x < 0.5 predict 0 else predict 1
            predict = model.calc_predict(output)
            # Number of correct predictions
            correct = predict.eq(target.view_as(predict)).long().sum().item()
            # Total number of needed predictions
            total = predict.view(-1).size(0)
            if correct == total : #or (self.grid_search.enabled and step > 1000):
                if not key in self.sols:
                    loss = model.calc_loss(output, target)
                    self.sols[key] = 0
                    self.solsSum[key] = 0
                    self.sols[key] += 1
                    self.solsSum[key] += step
                if correct == total:
                    self.print_stats(step, loss, correct, total, model)
                    print('{:.4f}'.format(float(self.solsSum[key])/self.sols[key]))
                break
            # calculate loss
            loss = model.calc_loss(output, target)
            # calculate deriviative of model.forward() and put it in model.parameters()...gradient
            loss.backward()
            # print progress of the learning
            # update model: model.parameters() -= lr * gradient
            optimizer.step()
            step += 1
        return step
    
    def print_stats(self, step, loss, correct, total, model):
        print("LR={}, HS={}, Number of layers={}, ActivOut={}, Step = {} Prediction = {}/{} Error = {}".format(
            model.learning_rate, model.hidden_size, model.layers_number, model.activation_hidden, step, correct, total, loss.item()))


Create data generator

In [0]:
class Limits:
    def __init__(self):
        self.time_limit = 2.0
        self.size_limit = 1000000
        self.test_limit = 1.0

class DataProvider:
    def __init__(self):
        self.number_of_cases = 10

    def get_index(self, tensor_index):
        index = 0
        for i in range(tensor_index.size(0)):
            index = 2*index + tensor_index[i].item()
        return index

    def calc_value(self, input_data, function_table, input_size, input_count_size):
        count = 0
        for i in range(input_count_size):
            count += function_table[self.get_index(input_data[i*input_size: (i+1)*input_size])].item()
        if count > input_count_size/2:
            return 1
        else:
            return 0

    def create_data(self, data_size, input_size, input_count_size, seed):
        torch.manual_seed(seed)
        function_size = 1 << input_size
        function_table = torch.ByteTensor(function_size).random_(0, 2)
        total_input_size = input_size*input_count_size
        data = torch.ByteTensor(data_size, total_input_size).random_(0, 2)
        target = torch.ByteTensor(data_size)
        for i in range(data_size):
            target[i] = self.calc_value(data[i], function_table, input_size, input_count_size)
        return (data.float().to(device), target.view(-1, 1).float().to(device))

    def create_case_data(self, case):
        input_size = 8
        data_size = (1<<input_size)*32
        input_count_size = case

        data, target = self.create_data(2*data_size, input_size, input_count_size, case)
        return sm.CaseData(case, Limits(), (data[:data_size], target[:data_size]), (data[data_size:], target[data_size:])).set_description("{} inputs per voter and {} voters".format(input_size, input_count_size))

class Config:
    def __init__(self):
        self.max_samples = 10000

    def get_data_provider(self):
        return DataProvider()

    def get_solution(self):
        return Solution()

Run training loop

In [0]:
# If you want to run specific case, put number here
sm.SolutionManager(Config()).run(case_number=2)

Best hyper parameters:

        self.learning_rate = 0.8
        self.momentum = 0.9
        self.hidden_size = 45
        self.layers_number = 5
        self.activation_hidden = 'relu'
        self.activation_output = 'sigmoid'